In [1]:
import io
import json
import random
import sys
import warnings
from pathlib import Path
from typing import List, Union

import pandas as pd
from seirsplus.models import *


In [134]:
data = pd.read_excel('Tumbe_Migration Data_Excel_Version 1.0_6th April 2020.xlsx', sheet_name= 'UP')
infections = pd.read_excel('Tumbe_Migration Data_Excel_Version 1.0_6th April 2020.xlsx', sheet_name= 'Covid19')

In [135]:
infections = infections[['City Name','Covid19 Cases','Population']].dropna()
infections['% infected'] = infections['Covid19 Cases']/infections['Population']

infections = infections[['City Name','% infected']]
infections = infections.transpose().reset_index(drop=True)

In [136]:
infections.columns = infections.iloc[0]
infections = infections.iloc[1:]

# infections

In [137]:
city_names = infections.columns.to_list()

In [138]:
for i in infections.columns:    
    
    infections = infections.rename(columns={i: "%_infected_"+i})

infections['key'] = 1

In [139]:
data['key'] = 1

data.shape
data = data.merge(infections, on='key', how='left')
data.shape

(70, 19)

(70, 29)

In [140]:
data['Inflow'] = data[city_names].sum(axis=1)

In [141]:
data['initI'] = 0

for i in city_names:
    
    data['initI'] = data['initI'] + data[i] * data['%_infected_'+i]


In [142]:
# data['%_travellers_infected'] = data['total']/data['Inflow']

In [143]:
data = data[['District Name','Total Population Person 2011','Inflow', 'initI']]

In [144]:
unknown_to_known_ratio = 50
data['initI'] = data['initI'] * unknown_to_known_ratio
data = data.rename(columns={'District Name':'district', 'Total Population Person 2011':'Population'})

data['initN'] = data['Population']

In [145]:
data = data[['district', 'Population', 'Inflow', 'initI',
       'initN']]

In [148]:
# data['initI'].sum()

### SEIRS Model

In [149]:
# hide
import io
def get_infections(
    initI: int = 100, initN: int = 10 ** 5, days_N: int = 21, beta: float = 2.4
) -> List[int]:
    model = SEIRSModel(
        beta=beta,
        sigma=1 / 5.2,
        gamma=1 / 12.39,
        initN=initN,
        initI=initI,
        beta_D=0.000,
        mu_D=0.02,
        #         sigma_D=1 / 4.0,
        #         gamma_D=1 / 9.0,
        theta_E=0.0002,
        theta_I=0.002,
        psi_E=0.2,
        psi_I=1.0,
    )
    model.run(T=days_N)
    S = model.numS  # time series of S counts
    E = model.numE  # time series of E counts
    I = model.numI  # time series of I counts
    D_E = model.numD_E  # time series of D_E counts
    D_I = model.numD_I  # time series of D_I counts
    R = model.numR  # time series of R counts
    F = model.numF  # time series of F counts
    t = model.tseries  # time values corresponding to the above time series
    return {"detected_exposed": D_E, "detected_infected": D_I, "model": model, "t": t}

In [150]:
def get_risk_estimates(
    assam_estimated_df: pd.DataFrame, beta: float
) -> Union[List, List]:
    days_N = 21
    atrisk_day14, atrisk_day21 = [], []
    for row in assam_estimated_df[["initI", "initN", "district"]].iterrows():
        initI, initN, district = row[1][0], int(row[1][1]), row[1][2]
        #     print(type(initI), type(initN))
        infection_results = get_infections(
            initI=initI, initN=initN, days_N=days_N, beta=beta
        )
        detected_infected = infection_results["detected_infected"]
        day14 = int(14 * len(detected_infected) / days_N)
        case_count_day14 = int(infection_results["detected_infected"][day14])
        case_count_day21 = int(infection_results["detected_infected"][-1])
        atrisk_day14.append(case_count_day14)
        atrisk_day21.append(case_count_day21)
    return atrisk_day14, atrisk_day21, infection_results

In [151]:
data['initI'] = data['initI'].astype(int)
data[f"day14"],data[f"day21"],infection_results = get_risk_estimates(data, beta=2.4)

t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90
t = 20.90


In [152]:
data.to_excel('seirsplus_up_covid_trend_analysis.xlsx', index=0)

In [153]:
data.sort_values(by='day21', ascending=False)

,district,Population,Inflow,initI,initN,day14,day21
63,Jaunpur,4494204,140550,298,4494204,911,12998
60,Azamgarh,4613913,109012,232,4613913,720,11892
44,Allahabad,5954391,79533,157,5954391,499,11188
57,Gorakhpur,4440895,89145,165,4440895,519,9956
66,Varanasi,3676841,68885,130,3676841,410,8045
42,Pratapgarh,3209141,59500,120,3209141,377,7217
10,Bulandshahr,3499171,61478,105,3499171,333,7044
48,Sultanpur,3797117,51470,90,3797117,287,6726
54,Basti,2464464,62372,138,2464464,426,6652
11,Aligarh,3673889,50756,86,3673889,275,6462
